# Advanced Analytics - Assignment 3

This assignment aims to predict the validity of Wikipedia Edits. The goal is to build a model which can classify incoming (stream) Wikipedia edits as Safe, Unsafe or Vandal. This notebook starts off by giving an overview of the contents. 

## Overview of the Notebook

> **1.** Problem Description <br>
> **2.** Data Capture & Cleaning <br>
> **3.** Text Procession & Ancillary Features <br>
> **4.** Model Training & Prediction <br>
> **5.** Model Evaluation <br>


## 1. Problem Description

To start, we will take a closer look at the data and how it is constructed. 
The data that has been streamed is text-based and formatted as a JSON dictionary with the following keys:

<b>Title page:</b> Title of the Wikipedia page.

<b>Text_new:</b> Text after the edit.

<b>Text_old:</b> Text before the edit.

<b>Name_user:</b> The user that edited the page. If the user is registered, their user name will show. In case of an anonymous edit, the user will be identified by his/her IP address at the time of editing.

<b>Label:</b> Label is the target feature which we aim to predict. The possible values are 'safe', 'unsafe' or 'vandal'.

<b>Comment:</b> The editor is asked to summarize the changes that have been made to the page. It is considered good practice to provide a summary about the edit, however, the user is free to leave this field blank, therefore empty values can occur.

<b>URL_page:</b> The URL of the Wikipedia page that was edited.

## 2. Data Capture & Cleaning

First, data was captured from the streaming server. The data that will be used for training and testing was collected non-stop from DD/MM/YYY until DD/MM/YYYY. 

Secondly, the streamed data should be filtered. Many folders out of the streamed data do not contain any 'part files'. It is easiest to delete these empty folders before preprocessing the data and training the model, since they do not contain any useful data.

In [ ]:
# Code uit 'DeleteEmptyDirectory'

After that, the separate 'part-files' are loaded into a single dataframe on which the model can later be trained. In this single dataframe, the solely text-based data (formatted as a JSON dictionary) will be converted into a more tabular structure. The keys that were used in the text-based data will become the features, as presented in the upper row: (comment, label, name_user, text_new, text_old, title_page, url_page).

In [ ]:
# Code uit 'LB-MNB Part 1 Step 0: Loading the data into a single dataframe'

## 3. Text Processing & Ancillary Features


### 3.1 Steps in processing the text

 **STEP 1a.** Tokenization & Normalization <br>
 
The values for 'new_text' and 'old_text' are still plain text. Therefore, a tokenizer is needed which converts the text into (text_old, text_new) into a list of words (words_old, words_new). 
<br>

The regexTokenizer is used because of its additional functionality compared to the standard Tokenizer that is built into Spark. One of these additional functionalities is that the tokens will automatically be normalized (decapitalized).
<br>

Additionally, the tokens in both lists are counted, in order to possibly use this information further.

In [1]:
# Code LB-MNB step 1a

 **STEP 1b.** Text difference <br>
 
The goal is to determine the difference between old_text and new_text, so to find what has been modified on the Wikipedia webpage. The new feature 'diff_text' will be created. This feature shows the exact changes made to the text, omitting the part of the text that is the same before and after the edit (resp. old_text and new_text).

Several types of changes could occur:
 - Spelling changes: <i> misteries --> mysteries</i>
 - Grammar changes: <i> On the country side, I ride my bike --> I ride my bike on the country side </i>
 - Drastic changes, such as completely new text, deleted text, ... <br>

So the essence of a change will probably lie in either deleted words, newly added words, rearranged words and/or edited words.

&emsp; <b> User defined function </b>

The code calculated the difference between the input and output text. This is accomplished by defining a UDF and a seperate function arrayUdf(). The udf is called on two columns *'words_old'* and *'words_new'*. Next a lambda function is defined to iterate over each row of the two input columns. Within the udf is refered to another function arrayUdf() which requires two inputs: the two tokenized lists of words which will be used to compute the difference. The arrayUdf() function acts as an itermediary to call on a different function: text_difference(). The text_difference() function uses the unified_diff generator from the difflib package to return the deltas between two lists of strings.

Through experimentation with the unified_diff generator, we found that it was much easier to first tokenize the input and output text and then compute the difference between the two tokenized lists of words. This in contrast to passing the two texts (*'text_old'* and *'text_new'*) of the rdd's as input directly and then tokenizing this *'diff_text'*. Although the latter method might create less computational overhead due to less tokenization, the former method proves to be much more reliable to determine which words have been deleted and which words are new.

In [ ]:
# Code LB-MNB step 1b

 **STEP 2.** Stop-Word Removal <br>
 
Stop-words should be filtered out before any training can happen. These stop-words are words that are very common in natural language. <br>
<br>
<i> IS DIT EEN SOORT PACKAGE DAT JE GEBRUIKT HEBT VOOR DIE STOPWOORDEN? EN HIER DAN NOG WAT TYPISCHE VOOR ONZE DATA AAN TOEGEVOEGD ZOALS DIE HTTP, WWW, COM, ... </i>

<br>
This will leave us with the dataset '<i>words_clean</i>'.

In [ ]:
# Code LB-MNB step 2

 **STEP 3.** Stemming <br>
 
Several stemming algorithms exist, of which the Porter algorithm is one of the least aggressive ones. The Snowball stemmer is slightly more aggressive at stemming the tokenized words, while being less aggressive than the Lancaster algorithm. <br>
We started off by trying the Snowball stemmer since it seemed like a nice 'middle ground' between the other two stemming variants. However, after inspecting the results, this stemmer still turned out to be too aggressive, stemming words as 'XXX' to 'YYY'. <br>
Therefore, the chosen algorithm for stemming is the Porter algorithm.<br>

<i> WEET IEMAND NOG EEN VOORBEELD VAN WAT DE WOORDEN WAREN DIE NOG TE STRENG AFGEKAPT WERDEN? </i>

<br>
This will result in the dataset '<i>words_stemmed</i>'.

In [ ]:
# Code LB-MNB step 3

 **STEP 4.** Feature Vectorization (TF-IDF) <br>
  <i> https://spark.apache.org/docs/latest/mllib-feature-extraction.html#tf-idf </i> <br>

TF-IDF or *term frequency - inverse document frequency* is a method used to check the importance of a term to a document of the corpus.<br>

**TF:** HashingTF converts the set of terms into fixed-length feature vectors, while utilizing the hashing trick. Based on the indices, term frequencies are calculated. Only using TF would overestimate the importance of commonly used words in natural language. Ofcourse, after applying stop-word removal many of these common words will already have been removed, but for remaining words that are not considered as stop-words, the problem will still arise. If terms appear frequently over the whole corpus, this means that this term contains no specific information for the particular document. To avoid this overestimation, idfModel will be applied after TF.<br>

**IDF:** idfModel takes the feature vectors created in HashinTF and scales them, so that terms which appear more frequently in the corpus will get a lower weight.<br>

The result we obtain is a numerical measure that shows how much information a term contains.

In [ ]:
# Code LB-MNB step 4

 **STEP 5.** String indexer <br>
 
In this final step the labels (*Safe, Unsafe and Vandal*) are encoded to label indices. The most frequent label gets index 0 while the least frequent label gets the last index depending on the number of indices. In this case the least frequent label gets index 2.

In our data 0 corresponds to *Safe*, 1 corresponds to *Unsafe*, and 2 corresponds to *Vandal*.

In [ ]:
# Code LB-MNB step 5

 **Assembly of Preprocessing Steps** <br>

In [ ]:
# LB-MNB step 6: Assembly of the preprocessing steps

### 3.2 Ancillary Features (a.k.a. Metadata Features):

Apart from the Bayesian classification model, it is usefull to manually declare some rules that are related to how humans recognize vandalism and to develop features for these rules that can be put in a model. <br>

After manually inspecting some *unsafe* and *vandal* part-files and after research, the following ancillary features have been found useful: <br>

 - Summary Comment Present? <br>
 
*IK GELOOF DAT WE TOT NU TOE ENKEL NOG MAAR FEATURES HEBBEN DIE NAAR DE OLD_TEXT, NEW_TEXT EN DIFF_TEXT KIJKEN, DUS DEZE HEBBEN WE NOG NIET, MISSCHIEN NUTTIG OM IN DE TOEKOMST TOCH OOK NOG TOE TE VOEGEN*


 - Comment Length <br>

*IDEM*


 - Empty Revision? <br>

*IDEM*


 - Character Repitition (e.g. booooobs) <br>

Longest sequence of the same character in an edit.


 - Size Ratio <br>

Size of the new text compared to the size of the new text.


 - Use of Emoticons / Bracket Ratio <br>



 - Alphanumeric Ratio <br>



 - Ratio Upper Case Letters to All Letters <br>



 - Username Registered? <br>

If the user is not registered, the IP address at the time of the edit will be shown. Most edits done by unregistered users are classified as *unsafe*.
*ZIE COMMENT SUMMARY COMMENT PRESENT*


 - Presence of Vulgar Words / Ratio <br>



 - Flamewars: Random Keystrokes <br>



 - Replacement Similarity <br>



## 4. Model Training & Prediction

### 4.1 Multinomial Bayesian Classifier

<i> UITLEG </i>

## 5. Model Evaluation